In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from sklearn.preprocessing import LabelBinarizer

In [2]:
def test(model, testdf):
    num_letters = {0: 'A', 1: 'B', 2: 'C', 3: 'D', 4: 'E', 5: 'F',
               6: 'G', 7: 'H', 8: 'I', 9: 'K',
               10: 'L', 11: 'M', 12: 'N', 13: 'O', 14: 'P',
               15: 'Q', 16: 'R', 17: 'S', 18: 'T', 19: 'U',
               20: 'V', 21: 'W', 22: 'X', 23: 'Y'}

    testdf = testdf.copy()

    y_test = testdf['label']
    del testdf['label']
    x_test = testdf.values
    x_test = x_test / 255
    x_test = x_test.reshape(-1, 70, 70, 3)

    lb = LabelBinarizer()
    y_test = lb.fit_transform(y_test)

    y_pred = model.predict(x_test)
    y_pred2 = np.array([np.argmax(y_pred[i]) for i in range(len(y_pred))])
    y_test2 = np.array([np.argmax(y_test[i]) for i in range(len(y_pred))])

    print('Accuracy: {}%'.format(round(np.sum(y_pred2 == y_test2)/len(y_pred2)*100, 2)))

In [3]:
M1 = keras.models.load_model('../models/M1.h5')
M2 = keras.models.load_model('../models/M2.h5')

In [5]:
print('Testing DS1_V')

DS1_V = pd.read_csv('../data/DS1_V.csv')
del DS1_V['Unnamed: 14701']

print('M1:')
test(M1, DS1_V)
print('M2:')
test(M2, DS1_V)

Testing DS1_V
M1:
Accuracy: 100.0%
M2:
Accuracy: 100.0%


In [6]:
print('Testing DS2_V')

DS2_V = pd.read_csv('../data/DS2_V.csv')
del DS2_V['Unnamed: 14701']

print('M1:')
test(M1, DS2_V)
print('M2:')
test(M2, DS2_V)

Testing DS2_V
M1:
Accuracy: 59.61%
M2:
Accuracy: 99.99%


In [7]:
print('Testing RW')

RW = pd.read_csv('../data/RW.csv')
del RW['Unnamed: 14701']

print('M1:')
test(M1, RW)
print('M2:')
test(M2, RW)

Testing RW
M1:
Accuracy: 18.66%
M2:
Accuracy: 28.51%
